In [21]:
from deta import Deta
from dotenv import load_dotenv
import os

load_dotenv()

# initialize with a project key
deta = Deta(os.environ.get("DETA_PROJECT_KEY"))

# create and use as many Drives as you want!
forecasts = deta.Drive("forecasts")

In [22]:
forecasts.list()

{'names': ['test.csv', 'verticaltbs.csv']}

In [16]:
import pandas as pd

In [17]:
df = pd.DataFrame([1,2,3,4,5])
df

,0
0,1
1,2
2,3
3,4
4,5


In [18]:
df.to_csv('test.csv')

In [19]:
forecasts.put('test.csv', path='test.csv')

'test.csv'

In [9]:
forecasts.list()

{'names': ['test.csv']}

In [20]:
forecasts.put('verticaltbs.csv', path='../forecasts/verticalTBS.csv')

'verticaltbs.csv'

In [13]:
forecasts.list()

{'names': ['test.csv', 'verticaltbs.csv']}

In [7]:
!pip install python-dotenv

In [12]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level, load
import comet_ml
import os
from dotenv import load_dotenv

load_dotenv()

comet_ml_api_key = os.environ.get("COMET_ML_API_KEY")
api = comet_ml.api.API(comet_ml_api_key)

api.download_registry_model("drdevinhopkins", "daily-visits",
                            version="1.0.0", output_path="../comet-ml-models", expand=True, stage=None)

set_log_level("ERROR")

loaded_model = load("../comet-ml-models/daily-visits.np")


COMET INFO: Downloading registry model 'daily-visits', version '1.0.0', stage None from workspace 'drdevinhopkins'...
COMET INFO: Unzipping model to '/Users/devinhopkins/Dropbox/Code/hourly-report-neuralprophet/comet-ml-models' ...
COMET INFO: done!


In [13]:

df = pd.read_csv(
    'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/daily-visits.csv')
df.ds = pd.to_datetime(df.ds)
df = df.sort_values(by='ds')


In [14]:

future = loaded_model.make_future_dataframe(df, periods=8)



forecast = loaded_model.predict(future)


ValueError: 
 you tried to log -1 which is currently not supported. Try a dict or a scalar/tensor.

In [ ]:

forecast_trimmed = forecast[forecast.y.isnull()].set_index('ds')
forecast_trimmed.to_csv('forecasts/daily_visits_raw.csv')

start = forecast_trimmed.index.tolist()[0]
forecast_length = len(forecast_trimmed.index.tolist())
i = 0
timestamp = pd.Timestamp.now().round('S').replace(tzinfo=None)
forecast_output_list = []
for ds in pd.date_range(start=start, periods=forecast_length, freq='D'):
    i = i+1
    forecast_output_list.append(
        {'ds': ds, 'visits': forecast_trimmed.loc[ds]['yhat'+str(i)], 'timestamp': timestamp})
forecast_output = pd.DataFrame(forecast_output_list)
forecast_output.to_csv('forecasts/daily_visits.csv', index=False)


In [9]:
!python ../scripts/create_daily_visits_model.py

COMET INFO: Experiment is live on comet.com https://www.comet.com/drdevinhopkins/daily-visits/0c8895e6816f44f28640ead1b0963728

Finding best initial lr: 100%|███████████████| 240/240 [00:02<00:00, 108.27it/s]
Missing logger folder: /Users/devinhopkins/Dropbox/Code/hourly-report-neuralprophet/notebooks/lightning_logs
Epoch 131: 100%|█| 131/131 [00:00<00:00, 156.75it/s, loss=0.0053, v_num=0, MAE=1
           MAE       RMSE      Loss  RegLoss  epoch
130  13.984174  18.121597  0.005373      0.0    130
Traceback (most recent call last):
  File "/Users/devinhopkins/Dropbox/Code/hourly-report-neuralprophet/notebooks/../scripts/create_daily_visits_model.py", line 50, in <module>
    save(m, "models/daily-visits.np")
  File "/Users/devinhopkins/anaconda3/envs/23-01-17/lib/python3.9/site-packages/neuralprophet/utils.py", line 44, in save
    torch.save(forecaster, path)
  File "/Users/devinhopkins/anaconda3/envs/23-01-17/lib/python3.9/site-packages/torch/serialization.py", line 422, in save
    